## Dependencies

In [1]:
# Import machine learning dependencies

import tensorflow as tf
import pandas as pd
from pathlib import Path
from sklearn import tree
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

import os
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.math import confusion_matrix as cm
from imblearn.over_sampling import SMOTE
from collections import Counter
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.under_sampling import ClusterCentroids

In [2]:
# Import other dependencies

from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime as dt
import pickle

In [3]:
# Python SQL toolkit and Object Relational Mapper

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

## Import Pickled File - From Anu's connection to SQL

In [4]:
# Open pickle
df = pickle.load(open('Resources/final_pickle.pkl', 'rb'))

In [5]:
df.head()

,loan_amount_000s,msamd,county_code,census_tract_number,applicant_income_000s,population,minority_population,hud_median_family_income,tract_to_msamd_income,number_of_owner_occupied_units,...,co_applicant_sex_Female,co_applicant_sex_Male,co_applicant_sex_No Co-Applicant,Region_MA,Region_MW,Region_NE,Region_NW,Region_SE,Region_SW,Region_W
11,92.0,33860.0,1.0,201.0,29.0,1948.0,12.58,59700.0,122.93,507.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
13,94.0,33860.0,1.0,201.0,53.0,1948.0,12.58,59700.0,122.93,507.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
18,147.0,33860.0,1.0,201.0,90.0,1948.0,12.58,59700.0,122.93,507.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
24,115.0,33860.0,1.0,201.0,30.0,1948.0,12.58,59700.0,122.93,507.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
33,160.0,33860.0,1.0,201.0,50.0,1948.0,12.58,59700.0,122.93,507.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [6]:
# Remove repetitive columns
df = df.drop(columns = ["owner_occupancy_Not Owner Occupied", "preapproval_Preapproval Not Requested", "applicant_ethnicity_Not Hispanic or Latino", ])
df.head()

,loan_amount_000s,msamd,county_code,census_tract_number,applicant_income_000s,population,minority_population,hud_median_family_income,tract_to_msamd_income,number_of_owner_occupied_units,...,co_applicant_sex_Female,co_applicant_sex_Male,co_applicant_sex_No Co-Applicant,Region_MA,Region_MW,Region_NE,Region_NW,Region_SE,Region_SW,Region_W
11,92.0,33860.0,1.0,201.0,29.0,1948.0,12.58,59700.0,122.93,507.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
13,94.0,33860.0,1.0,201.0,53.0,1948.0,12.58,59700.0,122.93,507.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
18,147.0,33860.0,1.0,201.0,90.0,1948.0,12.58,59700.0,122.93,507.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
24,115.0,33860.0,1.0,201.0,30.0,1948.0,12.58,59700.0,122.93,507.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
33,160.0,33860.0,1.0,201.0,50.0,1948.0,12.58,59700.0,122.93,507.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


### Run Neural Network

In [7]:
# Split our preprocessed data into our features and target arrays
y = df["action_taken_summary"].values
X = df.drop(["action_taken_summary"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

c:\Users\olivi\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [8]:
df['action_taken_summary'].value_counts()

1    147137
0     21803
Name: action_taken_summary, dtype: int64

In [9]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [10]:
# Use SMOTE to oversample the data
#X_resampled, y_resampled = SMOTE(random_state=42, sampling_strategy="auto").fit_resample(X_train_scaled, y_train)
#Counter(y_resampled)

In [11]:
# Use Random Oversampling on the data
#ros = RandomOverSampler(random_state=1)
#X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

In [12]:
# Use Random Undersampling on the data
#ros = RandomUnderSampler(random_state=42)
#X_resampled, y_resampled = ros.fit_resample(X_train_scaled, y_train)
#Counter(y_resampled)

In [13]:
# Use Cluster Centroid Undersampling on the data
cc = ClusterCentroids(random_state=42)
X_resampled, y_resampled = cc.fit_resample(X_train_scaled, y_train)
Counter(y_resampled)

MemoryError: Unable to allocate 13.4 GiB for an array with shape (110353, 16352) and data type float64

In [ ]:
import keras

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_resampled[0])
hidden_nodes_layer1 = 56
hidden_nodes_layer2 = 38

nn = tf.keras.models.Sequential()

#First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, kernel_initializer ='normal',input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2,kernel_initializer ='normal', activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

In [ ]:
# Compile the model
#nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
nn.compile(loss=keras.losses.BinaryCrossentropy(from_logits = False), optimizer=keras.optimizers.Adam(), metrics=[keras.metrics.BinaryAccuracy()])

In [ ]:
# Train the model
fit_model = nn.fit(X_resampled,y_resampled,epochs=5)

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_resampled,y_resampled,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
y_pred = (nn.predict(X_test_scaled) > 0.5).astype("int32")
y_pred

In [ ]:
matrix = cm(y_test,y_pred)
print(matrix)

In [ ]:
report = classification_report(y_test,y_pred)
print(report)